## Prueba asistente Extractor información de ofertas laborales a JSON

#### Aqui probamos que tal funciona el asistente que hemos creado y entrenado en la API de [OpenAI developer platform](https://platform.openai.com/) con las siguientes instrucciones de sistema:

<div style='background-color: #edf6f9; padding: 5px 10px; border-radius: 14px;'>
Vas a recibir descripciones de ofertas laborales relacionadas con el mundo del data. Tu tarea es devolver exclusivamente, sin añadir explicaciones ni texto adicional, un diccionario de Python con las siguientes claves y valores:

{
    "exp_min": (experiencia mínima requerida. Extrae este dato en formato de texto, ejemplo: '+3 años', '-2 años'. Si no está especificado, escribe 'desconocido'),
    "tipo_contrato": (tipo de contrato especificado, ejemplo: 'indefinido', 'temporal'. Si no está indicado, escribe 'Desconocido'),
    "estudios_min": (lista de Python con los estudios mínimos requeridos, estandarizados en minúsculas y sin tildes. Ejemplo: si aparece 'formación profesional superior' o 'grado superior', interpreta ambos como 'grado superior'. Si no hay datos, escribe ['desconocido']),
    "skills": (lista de Python con las tecnologías o habilidades técnicas mencionadas en la oferta. Normaliza y estandariza el formato: por ejemplo, 'PoweBi' y 'Power Bi' se convierten en 'powerbi' o 'natural language processing' se convierte en 'nlp' pero no uses dos notaciones para la misma tecnología. Si no hay datos, escribe ['desconocido'])
}

Pautas adicionales:
Limpieza de texto:

Elimina tildes en los valores antes de devolverlos.
Convierte todos los valores a minúsculas para mantener consistencia.
Estandarización:

En estudios_min y skills, normaliza las variantes que se refieran al mismo concepto bajo un único formato común.
Comportamiento con datos faltantes:

Si no puedes identificar un valor para alguna clave, utiliza 'desconocido' o ['desconocido'] según corresponda.
Enfoque tecnológico:

En skills, prioriza habilidades técnicas específicas del mundo del data (ejemplo: lenguajes de programación, herramientas de visualización, frameworks, tecnologías relacionadas con datos), pero también

Estrictamente estructurado:

Devuelve el diccionario con la estructura y formato exacto especificados.
</div>

<h3>Modelo utilizado: gpt-4o-2024-11-20</h3> (ya que nos permite formatear su salida de forma segura con un json schema)

### Json_Schema Indicado
```python
{
  "name": "job_description_schema",
  "strict": true,
  "schema": {
    "type": "object",
    "properties": {
      "exp_min": {
        "type": "string",
        "description": "The minimum experience required for the job, expressed in years. For example, '3 years' or '2-5 years'."
      },
      "tipo_contrato": {
        "type": "string",
        "description": "The type of contract offered for the job position. Examples: 'full-time', 'part-time', 'contract', etc."
      },
      "estudios_min": {
        "type": "array",
        "description": "List of minimum educational qualifications required for the job. Ensure degrees are standardized to a common naming convention (e.g., 'Bachelor's Degree', 'Master's Degree', 'PhD').",
        "items": {
          "type": "string",
          "description": "Degree type or level of education. For example, 'Bachelor's Degree in Computer Science'."
        }
      },
      "skills": {
        "type": "array",
        "description": "List of key technological skills required for the job. The values should be normalized and cleaned to ensure consistency, grouping variations of the same skill under a standardized format (e.g., 'Power BI', 'PoweBi' -> 'powerbi').",
        "items": {
          "type": "string",
          "description": "Name of a technology or skill relevant to the job. Normalize variations to a consistent format. Examples: 'python', 'sql', 'powerbi', etc."
        }
      }
    },
    "required": [
      "exp_min",
      "tipo_contrato",
      "estudios_min",
      "skills"
    ],
    "additionalProperties": false
  }
}
```


### cargamos librerias y variables de entorno

In [1]:
from dotenv import load_dotenv
import os
import openai
from openai import OpenAI
from typing import Optional, Dict, Any, List
import ast

load_dotenv()

client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

In [2]:
data_assistant = client.beta.assistants.retrieve(os.getenv('ASSISTANT_API_KEY'))

thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
thread_id=thread.id,
role="user",
content='''Acerca del rol

En Hybrid Intelligence buscamos un Data Scientist para unirse al equipo. Trabajarás en un equipo de expertos con los que podrás aprender y desarrollarte profesionalmente, mientras participas en el crecimiento y expansión del equipo. Como miembro del equipo de Hybrid Intelligence de Capgemini Engineering, trabajarás en una variedad de proyectos innovadores que marcan la diferencia. Nuestro trabajo es una fusión de matemáticas aplicadas, computación científica, inteligencia artificial e ingeniería de software.

¿Qué buscamos?

PhD, graduados en matemáticas, físicas, ingenierías, informática o campo relacionado.
Experiencia previa de al menos 2 años como Data Scientist.
Excelentes habilidades de programación (imprescindibles) R, Python (pandas, numpy, matplotlib, scikit-learn), experiencia en programación orientada a objetos y pySpark.
Experiencia llevando a cabo proyectos que involucren algoritmos de Machine Learning, idealmente en un entorno cloud con computación distribuida.
Sólidos fundamentos en estadística y teoría de la probabilidad para pruebas de hipótesis, análisis de regresión y diseño experimental.
Destreza en el manejo de datos, incluyendo limpieza, preprocesamiento y transformación de datos brutos en formatos utilizables para análisis y modelado.
Se valorará la experiencia desarrollando modelos matemáticos y algoritmos predictivos (optimización, modelos supervisados y no supervisados, clustering, redes neuronales...), Deep Learning, MLOPs.
Valorable conocimientos en natural language processing (NLP) y Large Language Models (LLMs).
Dominio del inglés y el español (B2-C1) y gran capacidad de comunicación.
Ubicación: Valladolid

Y nosotros ...¿Qué podemos ofrecerte?

Contrato indefinido.
Plan de carrera
Capgemini University y Mylearning con multitud de formaciones, además de acceso a la plataforma Udemy, Coursera, Pluralsight, Harvard Manager Mentor, entre otros.
Programa de certificaciones.
Formación en idiomas de inglés, francés y alemán con Education First (EF).
Participación en la Comunidad Internacional de DATA & AI.
Seguro de Vida y Accidentes.
Ticket restaurante o subvención de comida.
Plan de Compensación Flexible (seguro médico, guardería, transporte, formación).

La diversidad es clave de nuestra estrategia

Nuestro compromiso con la inclusión e igualdad de oportunidades hace que tengamos un Plan de Igualdad y un Código Ético que garantizan el desarrollo profesional de la plantilla y la igualdad de oportunidades en su selección dentro de un entorno libre de discriminación por cuestión de etnia, nacionalidad, origen social, edad, orientación sexual, expresión de género, religión o cualquier otra circunstancia personal, física o social.

En Capgemini Engineering valoramos todas las candidaturas, ¡no dudes en escribirte!'''
)

run = client.beta.threads.runs.create_and_poll(
thread_id=thread.id,
assistant_id=data_assistant.id
)


if run.status == 'completed': 
    messages = client.beta.threads.messages.list(
    thread_id=thread.id
    )
    print(messages)
else:
    print(run.status)

SyncCursorPage[Message](data=[Message(id='msg_9FJ0ov2tshS2baRnDhGnabpD', assistant_id='asst_F0QLSmqmosnyOEjfytVL3gGc', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='{"exp_min":"2 años","tipo_contrato":"indefinido","estudios_min":["phd","matematicas","fisicas","ingenierias","informatica","campo relacionado"],"skills":["r","python","pandas","numpy","matplotlib","scikit-learn","pythonspark","machine learning","cloud computing","distributed computing","estadistica","probabilidad","data preprocessing","mathematical modeling","deep learning","mlops","optimization","supervised learning","unsupervised learning","clustering","neural networks","nlp","llms"]}'), type='text')], created_at=1737720911, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_i9h4RPxm4BYfg1Lo7Gd6DV7w', status=None, thread_id='thread_x8hkvmmC3WHMti3hcedmgE2E'), Message(id='msg_4GqGGq1ti2rQG3v5NdkSTS3B', assistant_id

In [3]:
ast.literal_eval(messages.data[0].content[0].text.value)

{'exp_min': '2 años',
 'tipo_contrato': 'indefinido',
 'estudios_min': ['phd',
  'matematicas',
  'fisicas',
  'ingenierias',
  'informatica',
  'campo relacionado'],
 'skills': ['r',
  'python',
  'pandas',
  'numpy',
  'matplotlib',
  'scikit-learn',
  'pythonspark',
  'machine learning',
  'cloud computing',
  'distributed computing',
  'estadistica',
  'probabilidad',
  'data preprocessing',
  'mathematical modeling',
  'deep learning',
  'mlops',
  'optimization',
  'supervised learning',
  'unsupervised learning',
  'clustering',
  'neural networks',
  'nlp',
  'llms']}

In [5]:
message = client.beta.threads.messages.create(
thread_id=thread.id,
role="user",
content='''Imprescindible:
- Experiencia mínima de 4 años como Data Scientist.
-Formación universitaria: Grado, Master y/o Doctorado en Estadística, Matemáticas, Física, Informática u otros estudios universitarios de carácter técnico (Ingeniería, Telecomunicaciones?)
-Conocimientos avanzados en SQL y Python, Spark/Pyspark.
- Experiencia en sector retail'''
)

run = client.beta.threads.runs.create_and_poll(
thread_id=thread.id,
assistant_id=data_assistant.id
)


if run.status == 'completed': 
    messages = client.beta.threads.messages.list(
    thread_id=thread.id
    )
    print(ast.literal_eval(messages.data[0].content[0].text.value))
else:
    print(run.status)

{'exp_min': '4 años', 'tipo_contrato': 'desconocido', 'estudios_min': ['grado', 'master', 'phd', 'estadistica', 'matematicas', 'fisica', 'informatica', 'ingenieria', 'telecomunicaciones'], 'skills': ['sql', 'python', 'spark', 'pyspark', 'retail']}


In [7]:
message = client.beta.threads.messages.create(
thread_id=thread.id,
role="user",
content='''Requisitos
Estudios mínimos
Máster
Experiencia mínima
Al menos 4 años
Requisitos mínimos
Imprescindible:
- Experiencia mínima de 4 años como Data Scientist.
-Formación universitaria: Grado, Master y/o Doctorado en Estadística, Matemáticas, Física, Informática u otros estudios universitarios de carácter técnico (Ingeniería, Telecomunicaciones?)
-Conocimientos avanzados en SQL y Python, Spark/Pyspark.
- Experiencia en sector retail

Descripción
En BETWEEN apostamos por el mejor talento dentro del sector tecnológico. Nos involucramos en una gran variedad de proyectos punteros, trabajando con las últimas tecnologías. Actualmente en BETWEEN contamos con un equipo de más de 800 personas.
Trabajamos en ámbitos como BI, IoT, Big Data e I+D. En el área de Operaciones implantamos proyectos de Service Desk, Infraestructuras IT y proyectos Cloud, entre otros.
Actualmente seleccionamos un/a DATA SCIENTIST para incorporarse de forma indefinida en uno de los equipos de I+D que Between tiene implantados en uno de nuestros principales clientes del sector retail.

¿Qué haré? Estas serán tus funciones principales:
- Plantear soluciones basadas en datos, utilizando machine learning y estadística descriptiva
- Evaluar la calidad de los modelos y ajuste de modelos, incluyendo modelos avanzados como redes neuronales, aprendizaje por refuerzo, nuevos modelos de IA generativa.
- Aplicar las mejores prácticas y estándares de calidad en el desarrollo de proyectos de AI/ML, utilizando herramientas y prácticas metodológicas como CI/CD, orquestación de modelos, MLOps...
- Participar en la revisión y documentación de código, y aplicar prácticas de calidad en todo el ciclo de vida del software.
- Desarrollar modelos de datos y aplicar técnicas matemáticas, estadística y de programación

Si eres BETWEENER obtendrás estos beneficios:
- Seguro médico con Sanitas.
- Dispondrás de oportunidades de formación y desarrollo profesional: Idiomas, cursos en Udemy, Certificaciones, Programa The Power Business School y Bootcamps especializados (Agile, DevOps, IA & Big Data, Ciberseguridad, etc.).
- Contarás con un equipo de Talento que te acompañará durante toda tu experiencia con nosotros. ¡Haremos un café una vez al mes! Así, mantendremos el contacto.
- Pondremos a tu disposición un amplio abanico de retos y proyectos acordes con tus objetivos personales y profesionales.
- Posibilidad de elegir cómo obtener parte de tu salario gracias a las ventajas fiscales de nuestra Retribución Flexible (Ticket restaurant, Ticket transporte, Ticket guardería).
- Descuentos para acceder a centros de Fitness con Gympass.
- Acceso a nuestro Club de Descuentos: Inspiring Benefits.
- En BETWEEN nos gusta mucho hacer networking, por lo que te invitaremos a participar en todos estos eventos ¡y más!: las BETWEEN Nights, torneos deportivos, Black Mamba Race, viernes de vermut, juegos de mesa, etc.
- Programa remunerado de recomendación de Talento.
- En BETWEEN, la igualdad de oportunidades es uno de nuestros valores. Nuestro compromiso es contratar al mejor talento independientemente de su religión, sexo, edad y personas con otras capacidades y promover su desarrollo profesional y personal.

Tipo de industria de la oferta
Servicios y tecnología de la información
Categoría
Informática y telecomunicaciones - ERP, CRM, Business Intelligence
Nivel
Empleado/a
Número de vacantes
1
Salario
Salario: 42.000€ - 54.000€ Bruto/año
Beneficios sociales
Teletrabajo
Seguro médico
Cheque restaurante'''
)

run = client.beta.threads.runs.create_and_poll(
thread_id=thread.id,
assistant_id=data_assistant.id
)


if run.status == 'completed': 
    messages = client.beta.threads.messages.list(
    thread_id=thread.id
    )
    print(ast.literal_eval(messages.data[0].content[0].text.value))
else:
    print(run.status)

{'exp_min': '4 años', 'tipo_contrato': 'indefinido', 'estudios_min': ['master', 'grado', 'phd', 'estadistica', 'matematicas', 'fisica', 'informatica', 'ingenieria', 'telecomunicaciones'], 'skills': ['sql', 'python', 'spark', 'pyspark', 'machine learning', 'estadistica descriptiva', 'neural networks', 'reinforcement learning', 'ia generativa', 'ci/cd', 'mlops', 'data modeling', 'programacion', 'retail']}


In [9]:
message = client.beta.threads.messages.create(
thread_id=thread.id,
role="user",
content='''Ubicación:
Alicante 

Resumen del Puesto:
En Level UP buscamos un Especialista en Automatización e Inteligencia Artificial con actitud proactiva y capacidad para identificar y resolver problemas creativamente. Esta persona será clave en la implementación de procesos automatizados y en el desarrollo de herramientas innovadoras que impulsen nuestro crecimiento.

Responsabilidades:
• Desarrollar e implementar automatizaciones para procesos internos, integrando plataformas como WhatsApp, el CRM actual, Make,…

• Explorar y trabajar con APIs de diferentes plataformas (por ejemplo, Meta) para optimizar flujos de trabajo.

• Colaborar con equipos multidisciplinarios para identificar oportunidades de automatización que generen impacto directo en los objetivos de negocio.

• Realizar pruebas y ajustar herramientas para garantizar su eficacia y adaptabilidad a las necesidades de la empresa.

Requisitos:
• Conocimientos básicos de programación, preferiblemente en Python.

• Familiaridad con herramientas de automatización y flujos de trabajo (no es necesaria experiencia específica en herramientas como Make, pero sí disposición para aprender).

• Actitud proactiva, curiosidad y capacidad para aprender nuevas tecnologías rápidamente.

• Habilidades para indagar y comprender el funcionamiento de APIs de diversas plataformas.

• Capacidad para trabajar tanto de forma autónoma como en equipo.

Deseable:
• Experiencia previa en el desarrollo de soluciones automatizadas.

• Conocimientos básicos en inteligencia artificial o modelos de lenguaje (por ejemplo, GPTs).

• Creatividad y enfoque disruptivo para generar ideas innovadoras.

Qué Ofrecemos:
• Trabajar en la escuela de negocios número uno de España, en cuanto a resultados probados para autónomos y pymes.

• Ambiente de trabajo dinámico, con oportunidades de aprendizaje y desarrollo profesional continuo.

• Flexibilidad horaria y posibilidad de trabajo remoto.

• Participación en proyectos innovadores con impacto real en el negocio.

SALARIO: a convenir según candidato.'''
)

run = client.beta.threads.runs.create_and_poll(
thread_id=thread.id,
assistant_id=data_assistant.id
)


if run.status == 'completed': 
    messages = client.beta.threads.messages.list(
    thread_id=thread.id
    )
    print(ast.literal_eval(messages.data[0].content[0].text.value))
else:
    print(run.status)

{'exp_min': 'desconocido', 'tipo_contrato': 'desconocido', 'estudios_min': ['desconocido'], 'skills': ['python', 'automatizacion', 'apis', 'inteligencia artificial', 'gpts']}
